In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pickle
import time


In [4]:

driver = webdriver.Chrome()

In [5]:
driver.maximize_window()

In [6]:
from helpers import short_sleep
driver.get('https://www.linkedin.com/login')
short_sleep()


⏳ Human sleep: 3.73 sec


In [7]:
from helpers import human_sleep, short_sleep
email = driver.find_element(By.ID, 'username')
email.send_keys("infoedu420@gmail.com")

password = driver.find_element(By.ID, 'password')
password.send_keys("ADeeL1021@@")

password.submit()

human_sleep()
short_sleep()
# Save cookies to a file
cookies = driver.get_cookies()
with open('cookies.pkl', 'wb') as file:
    pickle.dump(cookies, file)

print("Cookies saved successfully!")
human_sleep()

⏳ Human sleep: 8.16 sec
⏳ Human sleep: 6.19 sec
Cookies saved successfully!
⏳ Human sleep: 6.74 sec


In [ ]:
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# --- Start WebDriver ---
driver = webdriver.Chrome()
driver.get("https://www.linkedin.com")  # Must load domain before adding cookies
time.sleep(3)

# --- Load cookies ---
try:
    with open("cookies.pkl", "rb") as file:
        cookies = pickle.load(file)

    for cookie in cookies:
        # Selenium throws an error if cookie has 'sameSite' set incorrectly
        cookie.pop('sameSite', None)

        try:
            driver.add_cookie(cookie)
        except Exception as e:
            print("Skipping cookie:", cookie.get("name"), "Error:", e)

    print("Cookies loaded successfully!")

    # Refresh to apply cookies and become logged in
    driver.refresh()
    time.sleep(3)

except FileNotFoundError:
    print("No cookie file found. Please log in first and save cookies.")



In [8]:
import pandas as pd

# Read URLs from Excel file
excel_file = r'C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\linkedin_profiles.xlsx'

try:
    # Read Excel file
    df_urls = pd.read_excel(excel_file)
    
    # Extract URLs from 'Profile URL' column
    urls = df_urls['Profile URL'].tolist()
    

    
    print(f"✓ Successfully loaded {len(urls)} URLs from {excel_file}")
    print(f"\nFirst 5 URLs:")
    for i, url in enumerate(urls[:5], 1):
        print(f"{i}. {url}")
    
    print(f"\nTotal URLs to process: {len(urls)}")
    
except FileNotFoundError:
    print(f"✗ Error: File '{excel_file}' not found!")
    print("Please make sure the Excel file is in the same directory as your script.")
    urls = []
    
except KeyError as e:
    print(f"✗ Error: Column 'Profile URL' not found!")
    print(f"Available columns: {df_urls.columns.tolist()}")
    urls = []
    
except Exception as e:
    print(f"✗ Error reading Excel file: {str(e)}")
    urls = []

✓ Successfully loaded 500 URLs from C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\linkedin_profiles.xlsx

First 5 URLs:
1. https://www.linkedin.com/in/billrmcdermott?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABZE5fQB3jwciCKPx3w7RJg91eFUQkbCJxw
2. https://www.linkedin.com/in/sardar-ali-wattoo?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAGuPgABMGBj25ZU932tvDwc-6rRwuClgiM
3. https://www.linkedin.com/in/anupammittal007?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAMCzYcBj7k8LBrG8tu6vjLRWGWaQ9WFxoc
4. https://www.linkedin.com/in/michaelbogner?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABMMYT8BA0ME7VhX43FG4TR6SSdkne1OnmM
5. https://www.linkedin.com/in/adnan-ishrat-mirza-77348736?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAeLi4cB2BFtiDgi1GGkYh7OiiLlw6C-QgU

Total URLs to process: 500


In [ ]:
from bs4 import BeautifulSoup
import random
import time
import pandas as pd
from helpers import short_sleep, human_sleep
import os

output_file = 'linkedin_profiles.xlsx'

# Check if file exists and load existing data
if os.path.exists(output_file):
    df_existing = pd.read_excel(output_file, engine='openpyxl')
    print(f"✓ Loaded existing file with {len(df_existing)} profiles")
    
    # Convert to list of dictionaries for easier manipulation
    all_profiles = df_existing.to_dict('records')
    
    # Get already scraped URLs (where name is not N/A or ERROR)
    scraped_urls = set(df_existing[
        (df_existing['name'].notna()) & 
        (df_existing['name'] != 'N/A') & 
        (df_existing['name'] != 'ERROR')
    ]['url'].tolist())
    
    print(f"✓ Found {len(scraped_urls)} already scraped profiles")
else:
    print("✓ No existing file found, starting fresh")
    all_profiles = []
    scraped_urls = set()

# Filter URLs to only process unscraped ones
urls_to_process = [url for url in urls if url not in scraped_urls]

print(f"\nTotal URLs: {len(urls)}")
print(f"Already scraped: {len(scraped_urls)}")
print(f"To be scraped: {len(urls_to_process)}")
print(f"{'='*50}\n")

if len(urls_to_process) == 0:
    print("✓ All profiles already scraped!")
else:
    for idx, url in enumerate(urls_to_process, 1):
        try:
            print(f"\nProcessing profile {idx}/{len(urls_to_process)}: {url}")
            
            # Navigate to profile
            driver.get(url)
            
            human_sleep()
            
            # Parse page source
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'lxml')
            
            # Initialize profile data
            profile_data = {}
            profile_data['url'] = url
            
            name_element = soup.find('h2', {'class': '_3cd47253 _5bf0b581 _6eac37de _3268a99a _94265cb4 _89bf2af1 _8c42697e b1dc0833 ff3847e9 _0c747956'})
            
            if name_element:
                name = name_element.get_text().strip()
                profile_data['name'] = name
                human_sleep()
                
                print(f"✓ Name found: {name}")
            else:
                print("✗ Name not found - element may not be loaded yet")
                profile_data['name'] = 'N/A'
            
            # Add to results
            all_profiles.append(profile_data)
            short_sleep()
            
            # Save to Excel after each extraction
            df = pd.DataFrame(all_profiles)
            df.to_excel(output_file, index=False, engine='openpyxl')
            print(f"✓ Data saved to {output_file} ({len(all_profiles)} profiles)")
            
            human_sleep()
                
        except Exception as e:
            print(f"✗ Error processing {url}: {str(e)}")
            all_profiles.append({'url': url, 'name': 'ERROR'})
            
            # Save even on error
            df = pd.DataFrame(all_profiles)
            df.to_excel(output_file, index=False, engine='openpyxl')
            
            continue

    print(f"\n{'='*50}")
    print(f"Scraping complete! Processed {len(urls_to_process)} new profiles")
    print(f"Total profiles in file: {len(all_profiles)}")
    print(f"Final data saved to: {output_file}")
    print(f"{'='*50}")


Processing profile 1/500: https://www.linkedin.com/in/billrmcdermott?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABZE5fQB3jwciCKPx3w7RJg91eFUQkbCJxw
⏳ Human sleep: 7.54 sec
⏳ Human sleep: 6.51 sec
✓ Name found: Bill McDermott
⏳ Human sleep: 5.48 sec
✓ Data saved to linkedin_profiles.xlsx (1 profiles)
⏳ Human sleep: 9.13 sec

Processing profile 2/500: https://www.linkedin.com/in/sardar-ali-wattoo?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAGuPgABMGBj25ZU932tvDwc-6rRwuClgiM
⏳ Human sleep: 10.78 sec
⏳ Human sleep: 10.42 sec
✓ Name found: Sardar-Ali Wattoo
⏳ Human sleep: 5.53 sec
✓ Data saved to linkedin_profiles.xlsx (2 profiles)
⏳ Human sleep: 6.14 sec

Processing profile 3/500: https://www.linkedin.com/in/anupammittal007?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAMCzYcBj7k8LBrG8tu6vjLRWGWaQ9WFxoc
⏳ Human sleep: 8.03 sec
⏳ Human sleep: 11.64 sec
✓ Name found: Anupam Mittal
⏳ Human sleep: 4.03 sec
✓ Data saved to linkedin_profiles.xlsx (3 profiles)
⏳ Human sleep: 8.52 sec



KeyboardInterrupt: 

In [22]:
from bs4 import BeautifulSoup
import random
import time
import pandas as pd
from helpers import short_sleep, human_sleep

# Read existing Excel file
input_file = 'linkedin_profiles.xlsx'
output_file = 'linkedin_profiles.xlsx'

try:
    df_existing = pd.read_excel(input_file, engine='openpyxl')
    print(f"✓ Loaded existing file with {len(df_existing)} profiles")
    
    # Check if 'headline' column exists, if not create it
    if 'headline' not in df_existing.columns:
        df_existing['headline'] = 'N/A'
        print("✓ Added 'headline' column")
    
except FileNotFoundError:
    print(f"✗ File '{input_file}' not found!")
    exit()

# Get URLs from the existing dataframe
urls = df_existing['url'].tolist()

print(f"\nStarting to scrape headlines for {len(urls)} profiles...")
print(f"{'='*50}\n")

for idx, url in enumerate(urls, 1):
    try:
        print(f"Processing profile {idx}/{len(urls)}: {url}")
        
        # Navigate to profile
        driver.get(url)
        human_sleep()
        
        # Parse page source
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        
        # Extract headline
        headline_element = soup.find('div', {'class': 'text-body-medium break-words'})
        
        if headline_element:
            headline = headline_element.get_text().strip()
            df_existing.at[idx-1, 'headline'] = headline
            print(f"✓ Headline found: {headline}")
        else:
            print("✗ Headline not found - element may not be loaded yet")
            df_existing.at[idx-1, 'headline'] = 'N/A'
        
        # Save to Excel after each extraction
        short_sleep()
        df_existing.to_excel(output_file, index=False, engine='openpyxl')
        print(f"✓ Data saved to {output_file}")
        
        short_sleep()
        
    except Exception as e:
        print(f"✗ Error processing {url}: {str(e)}")
        df_existing.at[idx-1, 'headline'] = 'ERROR'
        
        # Save even on error
        df_existing.to_excel(output_file, index=False, engine='openpyxl')
        continue

print(f"\n{'='*50}")
print(f"Scraping complete! Processed {len(urls)} profiles")
print(f"Final data saved to: {output_file}")
print(f"{'='*50}")

# Display summary
print("\nSummary:")
print(f"- Total profiles: {len(df_existing)}")
print(f"- Headlines found: {len(df_existing[df_existing['headline'].notna() & (df_existing['headline'] != 'N/A') & (df_existing['headline'] != 'ERROR')])}")
print(f"- Headlines not found: {len(df_existing[df_existing['headline'] == 'N/A'])}")
print(f"- Errors: {len(df_existing[df_existing['headline'] == 'ERROR'])}")

✓ Loaded existing file with 51 profiles
✓ Added 'headline' column

Starting to scrape headlines for 51 profiles...

Processing profile 1/51: https://www.linkedin.com/in/billrmcdermott?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABZE5fQB3jwciCKPx3w7RJg91eFUQkbCJxw
⏳ Human sleep: 10.7 sec
✓ Headline found: Chairman and CEO at ServiceNow
⏳ Human sleep: 3.41 sec
✓ Data saved to linkedin_profiles.xlsx
⏳ Human sleep: 6.66 sec
Processing profile 2/51: https://www.linkedin.com/in/sardar-ali-wattoo?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAGuPgABMGBj25ZU932tvDwc-6rRwuClgiM
⏳ Human sleep: 10.78 sec
✓ Headline found: Founder & CEO @ IEL | Financial Services Expertise
⏳ Human sleep: 6.08 sec
✓ Data saved to linkedin_profiles.xlsx
⏳ Human sleep: 7.47 sec
Processing profile 3/51: https://www.linkedin.com/in/anupammittal007?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAMCzYcBj7k8LBrG8tu6vjLRWGWaQ9WFxoc
⏳ Human sleep: 6.82 sec
✓ Headline found: Founder & CEO @ People Group | Tech & D2C Bu

In [9]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

def extract_about(soup):
    """Extract About section text from LinkedIn profile HTML"""
    
    # 🔵 1) Locate the About section
    about_section = None
    
    # Method 1: Find by ID anchor
    about_anchor = soup.find("div", id="about")
    if about_anchor:
        about_section = about_anchor.find_parent("section")
    
    # Method 2: Find by header text
    if not about_section:
        about_header = soup.find("h2", class_="pvs-header__title")
        if about_header and "About" in about_header.get_text():
            about_section = about_header.find_parent("section")
    
    # Method 3: Find section containing "About" text
    if not about_section:
        all_sections = soup.find_all("section", class_="artdeco-card")
        for section in all_sections:
            h2 = section.find("h2")
            if h2 and "About" in h2.get_text():
                about_section = section
                break
    
    if not about_section:
        print("⚠ About section not found")
        return ""
    
    # 🔵 2) Extract text - LinkedIn uses both visible and hidden spans
    # The visually-hidden span often contains the full text
    
    # Try to find the visually-hidden span first (full text)
    visually_hidden = about_section.find("span", class_="visually-hidden")
    if visually_hidden:
        text = visually_hidden.get_text(separator=" ", strip=True)
        text = " ".join(text.split())
        if len(text) > 40:
            return text
    
    # Try aria-hidden span (visible text, might be truncated)
    text_container = about_section.find("div", class_="inline-show-more-text")
    if text_container:
        aria_span = text_container.find("span", attrs={"aria-hidden": "true"})
        if aria_span:
            text = aria_span.get_text(separator=" ", strip=True)
            text = " ".join(text.split())
            if len(text) > 40:
                return text
    
    # 🔵 3) Fallback: Extract from display-flex div
    display_div = about_section.find("div", class_="display-flex ph5 pv3")
    if display_div:
        # Remove buttons and links
        for unwanted in display_div.find_all(["button", "a", "svg"]):
            unwanted.decompose()
        
        text = display_div.get_text(separator=" ", strip=True)
        text = " ".join(text.split())
        if len(text) > 40:
            return text
    
    # 🔵 4) Last resort: get all text from section
    about_copy = BeautifulSoup(str(about_section), 'lxml')
    for element in about_copy.find_all(["button", "a", "svg", "h2"]):
        element.decompose()
    
    text = about_copy.get_text(separator=" ", strip=True)
    text = " ".join(text.split())
    
    return text if len(text) > 40 else ""


# ========== MAIN SCRIPT ==========
from helpers import scroll_page, human_sleep, short_sleep

# Configuration
input_file = 'linkedin_profiles.xlsx'
output_file = 'linkedin_profiles.xlsx'
DEBUG_MODE = True  # Set to False to reduce output

try:
    df_existing = pd.read_excel(input_file, engine='openpyxl')
    print(f"✓ Loaded existing file with {len(df_existing)} profiles")
    
    # Check if 'about' column exists, if not create it
    if 'about' not in df_existing.columns:
        df_existing['about'] = 'N/A'
        print("✓ Added 'about' column")
    
except FileNotFoundError:
    print(f"✗ File '{input_file}' not found!")
    exit()

# Get URLs from the existing dataframe
urls = df_existing['url'].tolist()

print(f"\nStarting to scrape about sections for {len(urls)} profiles...")
print(f"{'='*50}\n")

for idx, url in enumerate(urls, 1):
    try:
        print(f"\n{'='*50}")
        print(f"Processing profile {idx}/{len(urls)}")
        print(f"URL: {url}")
        print(f"{'='*50}")
        
        # Navigate to profile
        driver.get(url)
        human_sleep()
        
        # Scroll to ensure About section is loaded
        scroll_page(driver, duration=6)
        human_sleep()
        
        # Extract about section using improved function
        try:
            print("--- Extracting About Section ---")
            
            # Parse page source
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'lxml')
            
            # Use the improved extract_about function (removed driver parameter)
            about = extract_about(soup)
            
            if about:
                df_existing.at[idx-1, 'about'] = about
                print(f"✓ About section found: {len(about)} characters")
                print(f"   Preview: {about[:200]}{'...' if len(about) > 200 else ''}")
            else:
                print("✗ About section not found - profile may not have an about section")
                df_existing.at[idx-1, 'about'] = 'N/A'
                
        except Exception as e:
            print(f"✗ Error extracting About section: {str(e)}")
            if DEBUG_MODE:
                import traceback
                traceback.print_exc()
            df_existing.at[idx-1, 'about'] = 'N/A'
        
        # Save to Excel after each extraction
        df_existing.to_excel(output_file, index=False, engine='openpyxl')
        print(f"✓ Data saved to {output_file}")
        
        human_sleep()
            
    except Exception as e:
        print(f"✗ Error processing {url}: {str(e)}")
        if DEBUG_MODE:
            import traceback
            traceback.print_exc()
        df_existing.at[idx-1, 'about'] = 'ERROR'
        
        # Save even on error
        df_existing.to_excel(output_file, index=False, engine='openpyxl')
        continue

print(f"\n{'='*50}")
print(f"Scraping complete! Processed {len(urls)} profiles")
print(f"Final data saved to: {output_file}")
print(f"{'='*50}")

# Display summary
print("\nSummary:")
print(f"- Total profiles: {len(df_existing)}")
about_found = df_existing[(df_existing['about'].notna()) & 
                          (df_existing['about'] != 'N/A') & 
                          (df_existing['about'] != 'ERROR')]
print(f"- About sections found: {len(about_found)}")
print(f"- About sections not found: {len(df_existing[df_existing['about'] == 'N/A'])}")
print(f"- Errors: {len(df_existing[df_existing['about'] == 'ERROR'])}")

# Show some statistics
if len(about_found) > 0:
    avg_length = about_found['about'].str.len().mean()
    print(f"- Average about section length: {int(avg_length)} characters")

✓ Loaded existing file with 51 profiles

Starting to scrape about sections for 51 profiles...


Processing profile 1/51
URL: https://www.linkedin.com/in/billrmcdermott?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABZE5fQB3jwciCKPx3w7RJg91eFUQkbCJxw
⏳ Human sleep: 9.06 sec


KeyboardInterrupt: 

In [10]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

def extract_about(soup):
    """Extract About section text from LinkedIn profile HTML"""
    
    # 🔵 1) Locate the About section
    about_section = None
    
    # Method 1: Find by ID anchor
    about_anchor = soup.find("div", id="about")
    if about_anchor:
        about_section = about_anchor.find_parent("section")
    
    # Method 2: Find by header text
    if not about_section:
        about_header = soup.find("h2", class_="pvs-header__title")
        if about_header and "About" in about_header.get_text():
            about_section = about_header.find_parent("section")
    
    # Method 3: Find section containing "About" text
    if not about_section:
        all_sections = soup.find_all("section", class_="artdeco-card")
        for section in all_sections:
            h2 = section.find("h2")
            if h2 and "About" in h2.get_text():
                about_section = section
                break
    
    if not about_section:
        print("⚠ About section not found")
        return ""
    
    # 🔵 2) Extract text - LinkedIn uses both visible and hidden spans
    # The visually-hidden span often contains the full text
    
    # Try to find the visually-hidden span first (full text)
    visually_hidden = about_section.find("span", class_="visually-hidden")
    if visually_hidden:
        text = visually_hidden.get_text(separator=" ", strip=True)
        text = " ".join(text.split())
        if len(text) > 40:
            return text
    
    # Try aria-hidden span (visible text, might be truncated)
    text_container = about_section.find("div", class_="inline-show-more-text")
    if text_container:
        aria_span = text_container.find("span", attrs={"aria-hidden": "true"})
        if aria_span:
            text = aria_span.get_text(separator=" ", strip=True)
            text = " ".join(text.split())
            if len(text) > 40:
                return text
    
    # 🔵 3) Fallback: Extract from display-flex div
    display_div = about_section.find("div", class_="display-flex ph5 pv3")
    if display_div:
        # Remove buttons and links
        for unwanted in display_div.find_all(["button", "a", "svg"]):
            unwanted.decompose()
        
        text = display_div.get_text(separator=" ", strip=True)
        text = " ".join(text.split())
        if len(text) > 40:
            return text
    
    # 🔵 4) Last resort: get all text from section
    about_copy = BeautifulSoup(str(about_section), 'lxml')
    for element in about_copy.find_all(["button", "a", "svg", "h2"]):
        element.decompose()
    
    text = about_copy.get_text(separator=" ", strip=True)
    text = " ".join(text.split())
    
    return text if len(text) > 40 else ""


# ========== MAIN SCRIPT ==========
from helpers import scroll_page, human_sleep, short_sleep

# Configuration
input_file = 'linkedin_profiles.xlsx'
output_file = 'linkedin_profiles.xlsx'
DEBUG_MODE = True  # Set to False to reduce output

try:
    df_existing = pd.read_excel(input_file, engine='openpyxl')
    print(f"✓ Loaded existing file with {len(df_existing)} profiles")
    
    # Check if 'about' column exists, if not create it
    if 'about' not in df_existing.columns:
        df_existing['about'] = 'N/A'
        print("✓ Added 'about' column")
    
except FileNotFoundError:
    print(f"✗ File '{input_file}' not found!")
    exit()

# Get URLs from the existing dataframe
urls = df_existing['url'].tolist()

# Filter out already processed profiles
def is_processed(row):
    """Check if about section is already processed"""
    about_value = row['about']
    # Consider processed if:
    # - Not null/NaN
    # - Not 'N/A' (initial value)
    # - Has actual content (length > 40 characters)
    if pd.isna(about_value):
        return False
    if about_value in ['N/A', 'ERROR']:
        return False
    if isinstance(about_value, str) and len(about_value) > 40:
        return True
    return False

# Count already processed
processed_count = df_existing.apply(is_processed, axis=1).sum()
print(f"✓ Found {processed_count} already processed profiles")
print(f"✓ {len(urls) - processed_count} profiles remaining to process")

print(f"\nStarting to scrape about sections...")
print(f"{'='*50}\n")

processed_in_session = 0
skipped_in_session = 0

for idx, url in enumerate(urls, 1):
    try:
        # Check if this profile is already processed
        if is_processed(df_existing.iloc[idx-1]):
            print(f"⏭  Skipping profile {idx}/{len(urls)} - Already processed")
            print(f"   URL: {url}")
            skipped_in_session += 1
            continue
        
        print(f"\n{'='*50}")
        print(f"Processing profile {idx}/{len(urls)}")
        print(f"URL: {url}")
        print(f"{'='*50}")
        
        # Navigate to profile
        driver.get(url)
        human_sleep()
        
        # Scroll to ensure About section is loaded
        scroll_page(driver, duration=6)
        human_sleep()
        
        # Extract about section using improved function
        try:
            print("--- Extracting About Section ---")
            
            # Parse page source
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'lxml')
            
            # Use the improved extract_about function
            about = extract_about(soup)
            
            if about:
                df_existing.at[idx-1, 'about'] = about
                print(f"✓ About section found: {len(about)} characters")
                print(f"   Preview: {about[:200]}{'...' if len(about) > 200 else ''}")
                processed_in_session += 1
            else:
                print("✗ About section not found - profile may not have an about section")
                df_existing.at[idx-1, 'about'] = 'N/A'
                
        except Exception as e:
            print(f"✗ Error extracting About section: {str(e)}")
            if DEBUG_MODE:
                import traceback
                traceback.print_exc()
            df_existing.at[idx-1, 'about'] = 'N/A'
        
        # Save to Excel after each extraction
        df_existing.to_excel(output_file, index=False, engine='openpyxl')
        print(f"✓ Data saved to {output_file}")
        
        human_sleep()
            
    except Exception as e:
        print(f"✗ Error processing {url}: {str(e)}")
        if DEBUG_MODE:
            import traceback
            traceback.print_exc()
        df_existing.at[idx-1, 'about'] = 'ERROR'
        
        # Save even on error
        df_existing.to_excel(output_file, index=False, engine='openpyxl')
        continue

print(f"\n{'='*50}")
print(f"Scraping complete!")
print(f"Final data saved to: {output_file}")
print(f"{'='*50}")

# Display summary
print("\nSession Summary:")
print(f"- Profiles processed in this session: {processed_in_session}")
print(f"- Profiles skipped (already processed): {skipped_in_session}")
print(f"- Total profiles in file: {len(df_existing)}")

print("\nOverall Summary:")
about_found = df_existing[(df_existing['about'].notna()) & 
                          (df_existing['about'] != 'N/A') & 
                          (df_existing['about'] != 'ERROR') &
                          (df_existing['about'].str.len() > 40)]
print(f"- About sections found: {len(about_found)}")
print(f"- About sections not found: {len(df_existing[df_existing['about'] == 'N/A'])}")
print(f"- Errors: {len(df_existing[df_existing['about'] == 'ERROR'])}")

# Show some statistics
if len(about_found) > 0:
    avg_length = about_found['about'].str.len().mean()
    print(f"- Average about section length: {int(avg_length)} characters")

✓ Loaded existing file with 51 profiles
✓ Found 16 already processed profiles
✓ 35 profiles remaining to process

Starting to scrape about sections...

⏭  Skipping profile 1/51 - Already processed
   URL: https://www.linkedin.com/in/billrmcdermott?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABZE5fQB3jwciCKPx3w7RJg91eFUQkbCJxw
⏭  Skipping profile 2/51 - Already processed
   URL: https://www.linkedin.com/in/sardar-ali-wattoo?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAGuPgABMGBj25ZU932tvDwc-6rRwuClgiM
⏭  Skipping profile 3/51 - Already processed
   URL: https://www.linkedin.com/in/anupammittal007?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAMCzYcBj7k8LBrG8tu6vjLRWGWaQ9WFxoc
⏭  Skipping profile 4/51 - Already processed
   URL: https://www.linkedin.com/in/michaelbogner?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABMMYT8BA0ME7VhX43FG4TR6SSdkne1OnmM
⏭  Skipping profile 5/51 - Already processed
   URL: https://www.linkedin.com/in/adnan-ishrat-mirza-77348736?miniProfileUrn=urn%3

In [1]:
import pandas as pd
import json
import ast

# ------------------------------------------------------------
# Clean + fix malformed experience data
# ------------------------------------------------------------
def clean_experience_value(exp_value):
    """Fix JSON-like string even if slightly malformed."""
    try:
        return json.loads(exp_value)
    except:
        try:
            return ast.literal_eval(exp_value)
        except:
            return None


# ------------------------------------------------------------
# Fix swapped fields
# ------------------------------------------------------------
def normalize_experience_item(item):
    company = item.get("company_name", "")
    total = item.get("total_duration", "")

    if any(x in company.lower() for x in ["yr", "mo", "yrs", "mos", "to"]):
        company, total = total, company

    positions = item.get("positions", [])
    new_positions = []

    for p in positions:
        loc = p.get("location", "")
        dur = p.get("duration", "")

        if any(x in loc.lower() for x in ["yr", "mo", "mos"]):
            loc, dur = "", loc

        new_positions.append({
            "title": p.get("title", "") or "N/A",
            "duration": dur or "N/A",
            "location": loc or "N/A",
            "description": p.get("description", "") or "N/A"
        })

    return {
        "company_name": company or "N/A",
        "total_duration": total or "N/A",
        "positions": new_positions
    }


# ------------------------------------------------------------
# Flatten experience list for Excel
# ------------------------------------------------------------
def flatten_experience_for_excel(exp_list):
    flattened = []
    
    for item in exp_list:
        fixed_item = normalize_experience_item(item)
        company = fixed_item["company_name"]
        total_duration = fixed_item["total_duration"]

        for pos in fixed_item["positions"]:
            flattened.append({
                "company_name": company,
                "company_total_duration": total_duration,
                "position_title": pos["title"],
                "position_duration": pos["duration"],
                "location": pos["location"],
                "description": pos["description"]
            })

    return flattened


# ------------------------------------------------------------
# Load Excel
# ------------------------------------------------------------
input_file = "linkedin_profiles.xlsx"
df = pd.read_excel(input_file, engine="openpyxl")

flattened_rows = []

# ------------------------------------------------------------
# Process each row
# ------------------------------------------------------------
def safe_value(v):
    """Return N/A if placeholder, empty or NaN."""
    if pd.isna(v) or v in ["", None]:
        return "N/A"
    return v

for idx, row in df.iterrows():
    exp_raw = row.get("experience", "")

    if pd.isna(exp_raw) or exp_raw in ["", "[]", "ERROR", "TIMEOUT_ERROR"]:
        continue

    exp_data = clean_experience_value(exp_raw)
    if not exp_data:
        print(f"⚠ Bad JSON in row {idx}")
        continue

    flattened = flatten_experience_for_excel(exp_data)

    for exp_row in flattened:
        flattened_rows.append({
            "profile_url": safe_value(row.get("url", "")),
            "profile_name": safe_value(row.get("name", "")),
            "headline": safe_value(row.get("headline", "")),
            "about": safe_value(row.get("about", "")),
            "company_name": safe_value(exp_row["company_name"]),
            "company_total_duration": safe_value(exp_row["company_total_duration"]),
            "position_title": safe_value(exp_row["position_title"]),
            "position_duration": safe_value(exp_row["position_duration"]),
            "location": safe_value(exp_row["location"]),
            "description": safe_value(exp_row["description"]),
        })


# ------------------------------------------------------------
# Save output Excel
# ------------------------------------------------------------
output_file = r"C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\user_profiles.xlsx"
pd.DataFrame(flattened_rows).to_excel(output_file, index=False, engine="openpyxl")

print(f"✅ Saved cleaned experience to: {output_file}")
print(f"📌 Total records: {len(flattened_rows)}")


✅ Saved cleaned experience to: C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\user_profiles.xlsx
📌 Total records: 203


In [10]:
from bs4 import BeautifulSoup
import pandas as pd
import json
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from helpers import scroll_page, human_sleep, short_sleep
import time

# Read existing Excel file
input_file = 'linkedin_profiles.xlsx'
output_file = 'linkedin_profiles.xlsx'

try:
    df_existing = pd.read_excel(input_file, engine='openpyxl')
    print(f"✓ Loaded existing file with {len(df_existing)} profiles")
    
    # Check if 'education' column exists, if not create it
    if 'education' not in df_existing.columns:
        df_existing['education'] = None
        print("✓ Added 'education' column")
    
except FileNotFoundError:
    print(f"✗ File '{input_file}' not found!")
    exit()

# Get URLs from the existing dataframe
urls = df_existing['url'].tolist()

# Filter out already processed profiles
def is_processed(row):
    """Check if education is already processed"""
    edu_value = row['education']
    if pd.isna(edu_value):
        return False
    if edu_value in ['ERROR', 'TIMEOUT_ERROR']:
        return False
    # Check if it's a valid JSON with content
    if isinstance(edu_value, str) and edu_value not in ['[]', '']:
        try:
            data = json.loads(edu_value)
            return len(data) > 0
        except:
            return False
    return False

# Count already processed
processed_count = df_existing.apply(is_processed, axis=1).sum()
print(f"✓ Found {processed_count} already processed profiles")
print(f"✓ {len(urls) - processed_count} profiles remaining to process")

print(f"\nStarting to scrape education for profiles...")
print(f"{'='*50}\n")

processed_in_session = 0
skipped_in_session = 0

# Function to extract education data from main page
def get_edu_from_main_page(edu):
    """Extract education from main profile page (before clicking 'Show all')"""
    edu_dict = {}
    
    try:
        # Find all spans with visually-hidden class
        all_spans = edu.find_all('span', {'class': 'visually-hidden'})
        
        if len(all_spans) >= 2:
            # First span: school name
            school_name = all_spans[0].get_text().strip()
            
            # Second span could be degree or duration depending on structure
            # Check if there's a <span class="t-14 t-normal"> for degree
            degree_span = edu.find('span', {'class': 't-14 t-normal'})
            if degree_span:
                degree_hidden = degree_span.find('span', {'class': 'visually-hidden'})
                degree = degree_hidden.get_text().strip() if degree_hidden else ''
            else:
                degree = ''
            
            # Duration span (usually has class 't-14 t-normal t-black--light')
            duration_span = edu.find('span', {'class': 't-14 t-normal t-black--light'})
            if duration_span:
                duration_hidden = duration_span.find('span', {'class': 'visually-hidden'})
                duration = duration_hidden.get_text().strip() if duration_hidden else ''
            else:
                # Fallback to checking all_spans
                duration = all_spans[2].get_text().strip() if len(all_spans) > 2 else ''
            
            # Try to find activities/description in sub-components
            description = ''
            activities = ''
            
            sub_components = edu.find('div', {'class': 'tJngkRGaszdJdUiYzrWQJJlmwIKcnHOPojFE pvs-entity__sub-components'})
            if sub_components:
                # Look for description in inline-show-more-text div
                desc_div = sub_components.find('div', {'class': 'oMmwBlVPusnoaaKuVwSPfttBvfmVuxGXwuE'})
                if desc_div:
                    desc_span = desc_div.find('span', {'class': 'visually-hidden'})
                    if desc_span:
                        full_text = desc_span.get_text().strip()
                        # Check if it's activities or description
                        if 'Activities and societies:' in full_text:
                            activities = full_text
                        else:
                            description = full_text
            
            edu_dict['school_name'] = school_name
            edu_dict['degree'] = degree
            edu_dict['duration'] = duration
            edu_dict['description'] = description
            edu_dict['activities'] = activities
            
            return edu_dict
        
        return None
    
    except Exception as e:
        print(f"  ⚠ Error extracting education entry: {str(e)}")
        return None

def get_edu_from_detail_page(edu):
    """Extract education from detailed education page (after clicking 'Show all')"""
    edu_dict = {}
    
    try:
        # School name - in the bold link
        school_div = edu.find('div', {'class': 'display-flex align-items-center mr1 hoverable-link-text t-bold'})
        if school_div:
            school_span = school_div.find('span', {'class': 'visually-hidden'})
            school_name = school_span.get_text().strip() if school_span else ''
        else:
            return None
        
        # Degree - in span with class 't-14 t-normal'
        degree = ''
        degree_span = edu.find('span', {'class': 't-14 t-normal'})
        if degree_span:
            degree_hidden = degree_span.find('span', {'class': 'visually-hidden'})
            degree = degree_hidden.get_text().strip() if degree_hidden else ''
        
        # Duration - in span with class 't-14 t-normal t-black--light'
        duration = ''
        duration_span = edu.find('span', {'class': 't-14 t-normal t-black--light'})
        if duration_span:
            duration_wrapper = duration_span.find('span', {'class': 'pvs-entity__caption-wrapper'})
            if duration_wrapper:
                duration_hidden = duration_wrapper.find('span', {'class': 'visually-hidden'})
                duration = duration_hidden.get_text().strip() if duration_hidden else ''
        
        # Description and Activities from sub-components
        description_parts = []
        activities = ''
        grade = ''
        
        sub_components = edu.find('div', {'class': 'tJngkRGaszdJdUiYzrWQJJlmwIKcnHOPojFE pvs-entity__sub-components'})
        if sub_components:
            # Find all list items in sub-components
            sub_items = sub_components.find_all('li', {'class': 'qSmJsZEEgCwbSdWFbdafHacrdpfgholcwkY'})
            
            for item in sub_items:
                # Check for text content
                text_div = item.find('div', {'class': 'oMmwBlVPusnoaaKuVwSPfttBvfmVuxGXwuE'})
                if text_div:
                    text_span = text_div.find('span', {'class': 'visually-hidden'})
                    if text_span:
                        text_content = text_span.get_text().strip()
                        
                        # Categorize the content
                        if 'Activities and societies:' in text_content:
                            activities = text_content
                        elif 'Grade:' in text_content:
                            grade = text_content
                        else:
                            description_parts.append(text_content)
                
                # Also check for regular text spans
                regular_text = item.find('div', {'class': 'display-flex align-items-center t-14 t-normal t-black'})
                if regular_text:
                    text_span = regular_text.find('span', {'class': 'visually-hidden'})
                    if text_span:
                        text_content = text_span.get_text().strip()
                        # Check if it contains skills or other info
                        if text_content and text_content not in description_parts:
                            if 'Skills:' in text_content or 'Grade:' in text_content:
                                grade = text_content
                            else:
                                description_parts.append(text_content)
        
        # Combine all description parts
        description = ' | '.join(description_parts) if description_parts else ''
        
        edu_dict['school_name'] = school_name
        edu_dict['degree'] = degree
        edu_dict['duration'] = duration
        edu_dict['description'] = description
        edu_dict['activities'] = activities
        edu_dict['grade'] = grade
        
        return edu_dict
    
    except Exception as e:
        print(f"  ⚠ Error extracting detailed education: {str(e)}")
        return None

# Function to click "Show all education" button
def click_show_all_education(driver):
    try:
        show_all_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "navigation-index-see-all-education"))
        )
        
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", show_all_button)
        time.sleep(1)
        
        try:
            driver.execute_script("arguments[0].click();", show_all_button)
            print("  ✓ Clicked 'Show all education' button")
            human_sleep()
            return True
        except Exception as e:
            print(f"  ⚠ Click failed: {str(e)}")
            return False
        
    except (TimeoutException, NoSuchElementException):
        print("  ℹ No 'Show all education' button found")
        return False
    except Exception as e:
        print(f"  ⚠ Error with button: {str(e)}")
        return False

# Function to flatten education data for Excel
def flatten_education_for_excel(education_list):
    """Convert nested education JSON to flat rows for Excel"""
    flattened = []
    
    for edu in education_list:
        flattened.append({
            'school_name': edu.get('school_name', ''),
            'degree': edu.get('degree', ''),
            'duration': edu.get('duration', ''),
            'description': edu.get('description', ''),
            'activities': edu.get('activities', ''),
            'grade': edu.get('grade', '')
        })
    
    return flattened

for idx, url in enumerate(urls, 1):
    try:
        # Check if this profile is already processed
        if is_processed(df_existing.iloc[idx-1]):
            print(f"⏭  Skipping profile {idx}/{len(urls)} - Already processed")
            print(f"   URL: {url}")
            skipped_in_session += 1
            continue
        
        print(f"\n{'='*50}")
        print(f"Processing profile {idx}/{len(urls)}")
        print(f"URL: {url}")
        print(f"{'='*50}")
        
        # Navigate to profile
        driver.get(url)
        human_sleep()
        
        # Scroll to load education section
        scroll_page(driver, duration=6)
        human_sleep()
        
        # Try to click "Show all education" button
        button_clicked = click_show_all_education(driver)
        
        # Parse page source
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        
        education_data = []
        
        if button_clicked:
            # Extract from detailed page
            print("  → Extracting from detailed education page...")
            
            # Look for the detailed education list
            detail_items = soup.find_all('li', {'class': 'pvs-list__paged-list-item'})
            
            if detail_items:
                print(f"  ✓ Found {len(detail_items)} detailed education entries")
                
                for item in detail_items:
                    edu_data = get_edu_from_detail_page(item)
                    if edu_data:
                        education_data.append(edu_data)
                        print(f"    ✓ Extracted: {edu_data.get('school_name', 'N/A')}")
            else:
                print("  ⚠ No detailed items found, falling back to main page extraction")
                button_clicked = False
        
        if not button_clicked:
            # Extract from main profile page
            print("  → Extracting from main profile page...")
            
            # Find all sections
            sections = soup.find_all('section', {'class': 'artdeco-card pv-profile-card break-words mt2'})
            
            # Find education section
            education_section = None
            for sec in sections:
                if sec.find('div', {'id': 'education'}):
                    education_section = sec
                    break
            
            if education_section:
                # Find all education items
                education_items = education_section.find_all('li', {'class': 'artdeco-list__item'})
                print(f"  ✓ Found {len(education_items)} education entries on main page")
                
                for edu_item in education_items:
                    edu_data = get_edu_from_main_page(edu_item)
                    if edu_data:
                        education_data.append(edu_data)
                        print(f"    ✓ Extracted: {edu_data.get('school_name', 'N/A')}")
            else:
                print("  ✗ Education section not found")
        
        # Store as JSON string in dataframe
        if education_data:
            df_existing.at[idx-1, 'education'] = json.dumps(education_data, ensure_ascii=False)
            print(f"  ✓ Successfully extracted {len(education_data)} education entries")
            processed_in_session += 1
        else:
            df_existing.at[idx-1, 'education'] = json.dumps([])
            print("  ℹ No education data extracted")
        
        # Save to Excel after each extraction
        short_sleep()
        df_existing.to_excel(output_file, index=False, engine='openpyxl')
        print(f"  ✓ Data saved to {output_file}")
        
        short_sleep()
        
    except Exception as e:
        print(f"  ✗ Error processing {url}: {str(e)}")
        df_existing.at[idx-1, 'education'] = 'ERROR'
        
        # Save even on error
        df_existing.to_excel(output_file, index=False, engine='openpyxl')
        continue

print(f"\n{'='*50}")
print(f"Scraping complete!")
print(f"Final data saved to: {output_file}")
print(f"{'='*50}")

# Display summary
print("\nSession Summary:")
print(f"- Profiles processed in this session: {processed_in_session}")
print(f"- Profiles skipped (already processed): {skipped_in_session}")
print(f"- Total profiles in file: {len(df_existing)}")

print("\nOverall Summary:")
education_found = df_existing[df_existing.apply(is_processed, axis=1)]
print(f"- Education extracted: {len(education_found)}")
print(f"- No education found: {len(df_existing[df_existing['education'] == '[]'])}")
print(f"- Errors: {len(df_existing[df_existing['education'].isin(['ERROR', 'TIMEOUT_ERROR'])])}")

# ========== CREATE FLATTENED EDUCATION SHEET ==========
print("\n" + "="*50)
print("Creating flattened education data...")
print("="*50)

# Create a new dataframe for flattened education
flattened_rows = []

for idx, row in df_existing.iterrows():
    if pd.notna(row['education']) and row['education'] not in ['ERROR', 'TIMEOUT_ERROR', '[]', '']:
        try:
            edu_data = json.loads(row['education'])
            flattened = flatten_education_for_excel(edu_data)
            
            for edu_row in flattened:
                flattened_rows.append({
                    'profile_url': row.get('url', ''),
                    'profile_name': row.get('name', ''),
                    'school_name': edu_row['school_name'],
                    'degree': edu_row['degree'],
                    'duration': edu_row['duration'],
                    'description': edu_row['description'],
                    'activities': edu_row['activities'],
                    'grade': edu_row['grade']
                })
        except Exception as e:
            print(f"⚠ Error flattening education for row {idx}: {str(e)}")
            continue

if flattened_rows:
    df_flattened = pd.DataFrame(flattened_rows)
    
    # Save to separate Excel file
    flattened_output = 'linkedin_profiles_education_detailed.xlsx'
    df_flattened.to_excel(flattened_output, index=False, engine='openpyxl')
    print(f"✓ Flattened education data saved to: {flattened_output}")
    print(f"✓ Total education records: {len(flattened_rows)}")
else:
    print("⚠ No education data to flatten")

✓ Loaded existing file with 51 profiles
✓ Added 'education' column
✓ Found 0 already processed profiles
✓ 51 profiles remaining to process

Starting to scrape education for profiles...


Processing profile 1/51
URL: https://www.linkedin.com/in/billrmcdermott?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABZE5fQB3jwciCKPx3w7RJg91eFUQkbCJxw
⏳ Human sleep: 8.54 sec
⏳ Human sleep: 6.27 sec
  ✓ Clicked 'Show all education' button
⏳ Human sleep: 6.75 sec
  → Extracting from detailed education page...
  ✓ Found 3 detailed education entries
    ✓ Extracted: Northwestern University - Kellogg School of Management
    ✓ Extracted: The Wharton School
    ✓ Extracted: Dowling College
  ✓ Successfully extracted 3 education entries
⏳ Human sleep: 3.36 sec
  ✓ Data saved to linkedin_profiles.xlsx
⏳ Human sleep: 5.62 sec

Processing profile 2/51
URL: https://www.linkedin.com/in/sardar-ali-wattoo?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAGuPgABMGBj25ZU932tvDwc-6rRwuClgiM
⏳ Human sleep: 9.75 sec


In [15]:
import pandas as pd
import json
import ast

# ============================================================
# GENERIC CLEANING
# ============================================================

def clean_json_value(value):
    """Fix JSON-like strings even if slightly malformed."""
    try:
        return json.loads(value)
    except:
        try:
            return ast.literal_eval(value)
        except:
            return None

def safe_value(v):
    """Return N/A if placeholder, empty or NaN."""
    if pd.isna(v) or v in ["", None]:
        return "N/A"
    return v


# ============================================================
# EDUCATION NORMALIZATION
# ============================================================

def normalize_education_item(item):
    """Ensure all fields exist and are valid."""
    return {
        "school_name": item.get("school_name", "N/A"),
        "degree": item.get("degree", "N/A"),
        "duration": item.get("duration", "N/A"),
        "description": item.get("description", "N/A"),
        "activities": item.get("activities", "N/A"),
        "grade": item.get("grade", "N/A"),
    }


def flatten_education_for_excel(edu_list):
    flattened = []
    for edu in edu_list:
        fixed = normalize_education_item(edu)
        flattened.append(fixed)
    return flattened


# ============================================================
# EXPERIENCE NORMALIZATION
# ============================================================

def normalize_experience_item(item):
    """
    Fix swapped company_name / total_duration fields + fix location/duration mixups.
    """
    company = item.get("company_name", "")
    total = item.get("total_duration", "")

    # Fix swapped
    if any(x in company.lower() for x in ["yr", "mo", "yrs", "mos", "to"]):
        company, total = total, company

    positions = item.get("positions", [])
    new_positions = []

    for p in positions:
        loc = p.get("location", "")
        dur = p.get("duration", "")

        # Fix swapped location-duration
        if any(x in loc.lower() for x in ["yr", "mo", "mos"]):
            loc, dur = "", loc

        new_positions.append({
            "title": p.get("title", "") or "N/A",
            "duration": dur or "N/A",
            "location": loc or "N/A",
            "description": p.get("description", "") or "N/A"
        })

    return {
        "company_name": company or "N/A",
        "total_duration": total or "N/A",
        "positions": new_positions
    }


def flatten_experience_for_excel(exp_list):
    flattened = []
    for item in exp_list:
        fixed_item = normalize_experience_item(item)
        company = fixed_item["company_name"]
        total_duration = fixed_item["total_duration"]

        for pos in fixed_item["positions"]:
            flattened.append({
                "company_name": company,
                "company_total_duration": total_duration,
                "position_title": pos["title"],
                "position_duration": pos["duration"],
                "location": pos["location"],
                "description": pos["description"]
            })
    return flattened


# ============================================================
# LOAD MAIN EXCEL
# ============================================================

input_file = "linkedin_profiles.xlsx"
df = pd.read_excel(input_file, engine="openpyxl")

# ============================================================
# FLATTEN EXPERIENCE
# ============================================================

exp_flattened = []

for idx, row in df.iterrows():
    exp_raw = row.get("experience", "")
    if pd.isna(exp_raw) or exp_raw in ["", "[]", "ERROR", "TIMEOUT_ERROR"]:
        continue

    exp_data = clean_json_value(exp_raw)
    if not exp_data:
        print(f"⚠ Bad experience JSON in row {idx}")
        continue

    flat_list = flatten_experience_for_excel(exp_data)

    for e in flat_list:
        exp_flattened.append({
            "profile_url": safe_value(row.get("url")),
            "profile_name": safe_value(row.get("name")),
            "headline": safe_value(row.get("headline")),
            "about": safe_value(row.get("about")),
            "company_name": safe_value(e["company_name"]),
            "company_total_duration": safe_value(e["company_total_duration"]),
            "position_title": safe_value(e["position_title"]),
            "position_duration": safe_value(e["position_duration"]),
            "location": safe_value(e["location"]),
            "description": safe_value(e["description"]),
        })

exp_output = r"C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\user_profiles_experience.xlsx"
pd.DataFrame(exp_flattened).to_excel(exp_output, index=False, engine="openpyxl")

print(f"✅ Experience saved to: {exp_output}")
print(f"📌 Total experience records: {len(exp_flattened)}")


# ============================================================
# FLATTEN EDUCATION
# ============================================================

edu_flattened = []

for idx, row in df.iterrows():
    edu_raw = row.get("education", "")
    if pd.isna(edu_raw) or edu_raw in ["", "[]", "ERROR", "TIMEOUT_ERROR"]:
        continue

    edu_data = clean_json_value(edu_raw)
    if not edu_data:
        print(f"⚠ Bad education JSON in row {idx}")
        continue

    flat_list = flatten_education_for_excel(edu_data)

    for edu in flat_list:
        edu_flattened.append({
            "profile_url": safe_value(row.get("url")),
            "profile_name": safe_value(row.get("name")),
            "school_name": safe_value(edu["school_name"]),
            "degree": safe_value(edu["degree"]),
            "duration": safe_value(edu["duration"]),
            "description": safe_value(edu["description"]),
            "activities": safe_value(edu["activities"]),
            "grade": safe_value(edu["grade"]),
        })

edu_output = r"C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\user_profiles_education.xlsx"
pd.DataFrame(edu_flattened).to_excel(edu_output, index=False, engine="openpyxl")

print(f"✅ Education saved to: {edu_output}")
print(f"📌 Total education records: {len(edu_flattened)}")


✅ Experience saved to: C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\user_profiles_experience.xlsx
📌 Total experience records: 203
✅ Education saved to: C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\user_profiles_education.xlsx
📌 Total education records: 110


In [22]:
import pandas as pd
import json
import ast

# ============================================================
# GENERIC CLEANING
# ============================================================

def clean_json_value(value):
    """Fix JSON-like strings even if slightly malformed."""
    try:
        return json.loads(value)
    except:
        try:
            return ast.literal_eval(value)
        except:
            return None

def safe_value(v):
    """Return N/A if placeholder, empty or NaN."""
    if pd.isna(v) or v in ["", None]:
        return "N/A"
    return v


# ============================================================
# EDUCATION NORMALIZATION
# ============================================================

def normalize_education_item(item):
    """Ensure all fields exist and convert empty values to N/A."""
    return {
        "school_name": safe_value(item.get("school_name")),
        "degree": safe_value(item.get("degree")),
        "duration": safe_value(item.get("duration")),
        "description": safe_value(item.get("description")),
        "activities": safe_value(item.get("activities")),
        "grade": safe_value(item.get("grade")),
    }

def flatten_education_for_excel(edu_list):
    return [normalize_education_item(edu) for edu in edu_list]


# ============================================================
# EXPERIENCE NORMALIZATION
# ============================================================

def normalize_experience_item(item):
    """Fix swapped fields and location/duration mixups."""
    company = safe_value(item.get("company_name"))
    total = safe_value(item.get("total_duration"))

    # Fix swapped fields if company contains year/month patterns
    if any(x in str(company).lower() for x in ["yr", "mo", "yrs", "mos", "to"]):
        company, total = total, company

    new_positions = []
    for p in item.get("positions", []):
        loc = p.get("location", "") or ""
        dur = p.get("duration", "") or ""

        # Fix swapped location/duration
        if any(x in loc.lower() for x in ["yr", "mo", "mos"]):
            loc, dur = "", loc

        new_positions.append({
            "title": safe_value(p.get("title")),
            "duration": safe_value(dur),
            "location": safe_value(loc),
            "description": safe_value(p.get("description"))
        })

    return {
        "company_name": safe_value(company),
        "total_duration": safe_value(total),
        "positions": new_positions
    }

def flatten_experience_for_excel(exp_list):
    flattened = []
    for item in exp_list:
        fixed = normalize_experience_item(item)
        for pos in fixed["positions"]:
            flattened.append({
                "company_name": fixed["company_name"],
                "company_total_duration": fixed["total_duration"],
                "position_title": pos["title"],
                "position_duration": pos["duration"],
                "location": pos["location"],
                "description": pos["description"]
            })
    return flattened


# ============================================================
# LOAD MAIN FILE
# ============================================================

input_file = "linkedin_profiles.xlsx"
df = pd.read_excel(input_file, engine="openpyxl")


# ============================================================
# PROCESS EXPERIENCE
# ============================================================

exp_rows = []

for idx, row in df.iterrows():
    exp_raw = row.get("experience", "")
    if pd.isna(exp_raw) or exp_raw in ["", "[]", "ERROR", "TIMEOUT_ERROR"]:
        continue

    exp_data = clean_json_value(exp_raw)
    if not exp_data:
        print(f"⚠ Bad experience JSON in row {idx}")
        continue

    flat_list = flatten_experience_for_excel(exp_data)

    for e in flat_list:
        exp_rows.append({
            "profile_url": safe_value(row.get("url")),
            "profile_name": safe_value(row.get("name")),
            "headline": safe_value(row.get("headline")),
            "about": safe_value(row.get("about")),
            **e
        })

df_exp = pd.DataFrame(exp_rows)


# ============================================================
# PROCESS EDUCATION
# ============================================================

edu_rows = []

for idx, row in df.iterrows():
    edu_raw = row.get("education", "")
    if pd.isna(edu_raw) or edu_raw in ["", "[]", "ERROR", "TIMEOUT_ERROR"]:
        continue

    edu_data = clean_json_value(edu_raw)
    if not edu_data:
        print(f"⚠ Bad education JSON in row {idx}")
        continue

    flat_list = flatten_education_for_excel(edu_data)

    for edu in flat_list:
        edu_rows.append({
            "profile_url": safe_value(row.get("url")),
            "profile_name": safe_value(row.get("name")),
            **edu
        })

df_edu = pd.DataFrame(edu_rows)


# ============================================================
# SAVE BOTH TO ONE EXCEL
# ============================================================

output_file = r"C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\linkedin_profiles_all.xlsx"

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    df_exp.to_excel(writer, sheet_name="Experience", index=False)
    df_edu.to_excel(writer, sheet_name="Education", index=False)

print("===================================")
print(f"✅ ALL DATA SAVED TO ONE FILE:")
print(output_file)
print("📌 Experience rows:", len(df_exp))
print("📌 Education rows:", len(df_edu))
print("===================================")


✅ ALL DATA SAVED TO ONE FILE:
C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\linkedin_profiles_all.xlsx
📌 Experience rows: 203
📌 Education rows: 110


In [21]:
import pandas as pd
import json
import ast

# ============================================================
# GENERIC CLEANING
# ============================================================

def clean_json_value(value):
    """Fix JSON-like strings even if slightly malformed."""
    try:
        return json.loads(value)
    except:
        try:
            return ast.literal_eval(value)
        except:
            return None

def safe_value(v):
    """Return N/A if placeholder, empty or NaN."""
    if pd.isna(v) or v in ["", None]:
        return "N/A"
    return v


# ============================================================
# EDUCATION NORMALIZATION
# ============================================================

def normalize_education_item(item):
    """Ensure all fields exist and convert empty values to N/A."""
    return {
        "school_name": safe_value(item.get("school_name")),
        "degree": safe_value(item.get("degree")),
        "duration": safe_value(item.get("duration")),
        "description": safe_value(item.get("description")),
        "activities": safe_value(item.get("activities")),
        "grade": safe_value(item.get("grade")),
    }

def flatten_education_for_excel(edu_list):
    return [normalize_education_item(edu) for edu in edu_list]


# ============================================================
# EXPERIENCE NORMALIZATION
# ============================================================

def normalize_experience_item(item):
    """Fix swapped fields and location/duration mixups."""
    company = safe_value(item.get("company_name"))
    total = safe_value(item.get("total_duration"))

    if any(x in str(company).lower() for x in ["yr", "mo", "yrs", "mos", "to"]):
        company, total = total, company

    new_positions = []
    for p in item.get("positions", []):
        loc = p.get("location", "") or ""
        dur = p.get("duration", "") or ""

        if any(x in loc.lower() for x in ["yr", "mo", "mos"]):
            loc, dur = "", loc

        new_positions.append({
            "title": safe_value(p.get("title")),
            "duration": safe_value(dur),
            "location": safe_value(loc),
            "description": safe_value(p.get("description"))
        })

    return {
        "company_name": safe_value(company),
        "total_duration": safe_value(total),
        "positions": new_positions
    }

def flatten_experience_for_excel(exp_list):
    flattened = []
    for item in exp_list:
        fixed = normalize_experience_item(item)
        for pos in fixed["positions"]:
            flattened.append({
                "company_name": fixed["company_name"],
                "company_total_duration": fixed["total_duration"],
                "position_title": pos["title"],
                "position_duration": pos["duration"],
                "location": pos["location"],
                "description": pos["description"]
            })
    return flattened


# ============================================================
# LOAD MAIN FILE
# ============================================================

input_file = "linkedin_profiles.xlsx"
df = pd.read_excel(input_file, engine="openpyxl")


# ============================================================
# PROCESS EXPERIENCE
# ============================================================

combined_rows = []

for idx, row in df.iterrows():
    exp_raw = row.get("experience", "")
    if pd.isna(exp_raw) or exp_raw in ["", "[]", "ERROR", "TIMEOUT_ERROR"]:
        pass
    else:
        exp_data = clean_json_value(exp_raw)
        if exp_data:
            flat_list = flatten_experience_for_excel(exp_data)
            for e in flat_list:
                combined_rows.append({
                    "profile_url": safe_value(row.get("url")),
                    "profile_name": safe_value(row.get("name")),
                    "headline": safe_value(row.get("headline")),
                    "about": safe_value(row.get("about")),
                    "section": "Experience",
                    **e
                })

    # ============================================================
    # PROCESS EDUCATION
    # ============================================================

    edu_raw = row.get("education", "")
    if pd.isna(edu_raw) or edu_raw in ["", "[]", "ERROR", "TIMEOUT_ERROR"]:
        continue

    edu_data = clean_json_value(edu_raw)
    if not edu_data:
        print(f"⚠ Bad education JSON in row {idx}")
        continue

    flat_list = flatten_education_for_excel(edu_data)

    for edu in flat_list:
        combined_rows.append({
            "profile_url": safe_value(row.get("url")),
            "profile_name": safe_value(row.get("name")),
            "headline": safe_value(row.get("headline")),
            "about": safe_value(row.get("about")),
            "section": "Education",
            **edu
        })


# ============================================================
# CREATE ONE COMBINED DATAFRAME
# ============================================================

df_combined = pd.DataFrame(combined_rows)


# ============================================================
# SAVE ONE SHEET ONLY
# ============================================================

output_file = r"C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\linkedin_profiles_all.xlsx"

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    df_combined.to_excel(writer, sheet_name="All Data", index=False)

print("===================================")
print("✅ ALL DATA SAVED INTO ONE SHEET!")
print(output_file)
print("📌 Total rows:", len(df_combined))
print("===================================")


✅ ALL DATA SAVED INTO ONE SHEET!
C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\linkedin_profiles_all.xlsx
📌 Total rows: 313


In [23]:
import pandas as pd
import json
import ast

# ============================================================
# GENERIC CLEANING
# ============================================================

def clean_json_value(value):
    """Fix JSON-like strings even if slightly malformed."""
    try:
        return json.loads(value)
    except:
        try:
            return ast.literal_eval(value)
        except:
            return None

def safe_value(v):
    """Return N/A if placeholder, empty or NaN."""
    if pd.isna(v) or v in ["", None]:
        return "N/A"
    return v


# ============================================================
# EDUCATION NORMALIZATION
# ============================================================

def normalize_education_item(item):
    """Ensure all fields exist and convert empty values to N/A."""
    return {
        "school_name": safe_value(item.get("school_name")),
        "degree": safe_value(item.get("degree")),
        "duration": safe_value(item.get("duration")),
        "description": safe_value(item.get("description")),
        "activities": safe_value(item.get("activities")),
        "grade": safe_value(item.get("grade")),
    }

def flatten_education_for_excel(edu_list):
    return [normalize_education_item(edu) for edu in edu_list]


# ============================================================
# EXPERIENCE NORMALIZATION
# ============================================================

def normalize_experience_item(item):
    """Fix swapped fields and location/duration mixups."""
    company = safe_value(item.get("company_name"))
    total = safe_value(item.get("total_duration"))

    # Fix swapped fields if company contains year/month patterns
    if any(x in str(company).lower() for x in ["yr", "mo", "yrs", "mos", "to"]):
        company, total = total, company

    new_positions = []
    for p in item.get("positions", []):
        loc = p.get("location", "") or ""
        dur = p.get("duration", "") or ""

        # Fix swapped location/duration
        if any(x in loc.lower() for x in ["yr", "mo", "mos"]):
            loc, dur = "", loc

        new_positions.append({
            "title": safe_value(p.get("title")),
            "duration": safe_value(dur),
            "location": safe_value(loc),
            "description": safe_value(p.get("description"))
        })

    return {
        "company_name": safe_value(company),
        "total_duration": safe_value(total),
        "positions": new_positions
    }

def flatten_experience_for_excel(exp_list):
    flattened = []
    for item in exp_list:
        fixed = normalize_experience_item(item)
        for pos in fixed["positions"]:
            flattened.append({
                "company_name": fixed["company_name"],
                "company_total_duration": fixed["total_duration"],
                "position_title": pos["title"],
                "position_duration": pos["duration"],
                "location": pos["location"],
                "description": pos["description"]
            })
    return flattened


# ============================================================
# LOAD MAIN FILE
# ============================================================

input_file = "linkedin_profiles.xlsx"
df = pd.read_excel(input_file, engine="openpyxl")


# ============================================================
# PROCESS AND MERGE EXPERIENCE & EDUCATION
# ============================================================

merged_rows = []

for idx, row in df.iterrows():
    profile_url = safe_value(row.get("url"))
    profile_name = safe_value(row.get("name"))
    headline = safe_value(row.get("headline"))
    about = safe_value(row.get("about"))
    
    # Process Experience
    exp_raw = row.get("experience", "")
    exp_list = []
    
    if not (pd.isna(exp_raw) or exp_raw in ["", "[]", "ERROR", "TIMEOUT_ERROR"]):
        exp_data = clean_json_value(exp_raw)
        if exp_data:
            exp_list = flatten_experience_for_excel(exp_data)
        else:
            print(f"⚠ Bad experience JSON in row {idx}")
    
    # Process Education
    edu_raw = row.get("education", "")
    edu_list = []
    
    if not (pd.isna(edu_raw) or edu_raw in ["", "[]", "ERROR", "TIMEOUT_ERROR"]):
        edu_data = clean_json_value(edu_raw)
        if edu_data:
            edu_list = flatten_education_for_excel(edu_data)
        else:
            print(f"⚠ Bad education JSON in row {idx}")
    
    # Create cartesian product of experience and education
    if exp_list and edu_list:
        # Both experience and education exist
        for exp in exp_list:
            for edu in edu_list:
                merged_rows.append({
                    "profile_url": profile_url,
                    "profile_name": profile_name,
                    "headline": headline,
                    "about": about,
                    "company_name": exp["company_name"],
                    "company_total_duration": exp["company_total_duration"],
                    "position_title": exp["position_title"],
                    "position_duration": exp["position_duration"],
                    "location": exp["location"],
                    "experience_description": exp["description"],
                    "school_name": edu["school_name"],
                    "degree": edu["degree"],
                    "education_duration": edu["duration"],
                    "education_description": edu["description"],
                    "activities": edu["activities"],
                    "grade": edu["grade"]
                })
    elif exp_list:
        # Only experience exists
        for exp in exp_list:
            merged_rows.append({
                "profile_url": profile_url,
                "profile_name": profile_name,
                "headline": headline,
                "about": about,
                "company_name": exp["company_name"],
                "company_total_duration": exp["company_total_duration"],
                "position_title": exp["position_title"],
                "position_duration": exp["position_duration"],
                "location": exp["location"],
                "experience_description": exp["description"],
                "school_name": "N/A",
                "degree": "N/A",
                "education_duration": "N/A",
                "education_description": "N/A",
                "activities": "N/A",
                "grade": "N/A"
            })
    elif edu_list:
        # Only education exists
        for edu in edu_list:
            merged_rows.append({
                "profile_url": profile_url,
                "profile_name": profile_name,
                "headline": headline,
                "about": about,
                "company_name": "N/A",
                "company_total_duration": "N/A",
                "position_title": "N/A",
                "position_duration": "N/A",
                "location": "N/A",
                "experience_description": "N/A",
                "school_name": edu["school_name"],
                "degree": edu["degree"],
                "education_duration": edu["duration"],
                "education_description": edu["description"],
                "activities": edu["activities"],
                "grade": edu["grade"]
            })
    else:
        # Neither experience nor education exists
        merged_rows.append({
            "profile_url": profile_url,
            "profile_name": profile_name,
            "headline": headline,
            "about": about,
            "company_name": "N/A",
            "company_total_duration": "N/A",
            "position_title": "N/A",
            "position_duration": "N/A",
            "location": "N/A",
            "experience_description": "N/A",
            "school_name": "N/A",
            "degree": "N/A",
            "education_duration": "N/A",
            "education_description": "N/A",
            "activities": "N/A",
            "grade": "N/A"
        })

df_merged = pd.DataFrame(merged_rows)


# ============================================================
# SAVE MERGED DATA TO EXCEL
# ============================================================

output_file = r"C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\linkedin_profiles_merged.xlsx"

df_merged.to_excel(output_file, sheet_name="Merged Data", index=False, engine="openpyxl")

print("===================================")
print(f"✅ MERGED DATA SAVED TO:")
print(output_file)
print("📌 Total merged rows:", len(df_merged))
print("📌 Unique profiles:", df['url'].nunique())
print("===================================")

# Display column summary
print("\n📋 Columns in merged sheet:")
for col in df_merged.columns:
    print(f"   - {col}")

✅ MERGED DATA SAVED TO:
C:\Users\Usman Asghar\Desktop\Intern\LinkedIn Web Scraping\Excel File\linkedin_profiles_merged.xlsx
📌 Total merged rows: 569
📌 Unique profiles: 51

📋 Columns in merged sheet:
   - profile_url
   - profile_name
   - headline
   - about
   - company_name
   - company_total_duration
   - position_title
   - position_duration
   - location
   - experience_description
   - school_name
   - degree
   - education_duration
   - education_description
   - activities
   - grade
